In [ ]:
# python libraries for data visualization and manipulation
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style

# model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image

# preprocess.
from keras.preprocessing.image import ImageDataGenerator

#deep learning libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

# specifically for convolutional neural networks
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
 
import tensorflow as tf
import tensorflow_hub as hub
import random as rn

print("TF version:", tf.__version__)
print("TF Hub version:", hub.__version__)
print("GPU", "available (True)" if tf.config.list_physical_devices("GPU") else "not available :(")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
daisy_flower_dir = "/content/gdrive/MyDrive/flowers/daisy"
sunflower_dir = "/content/gdrive/MyDrive/flowers/sunflower"
tulip_flower_dir = "/content/gdrive/MyDrive/flowers/tulip"
dandelion_flower_dir = "/content/gdrive/MyDrive/flowers/dandelion"
rose_flower_dir =  "/content/gdrive/MyDrive/flowers/rose"

In [ ]:
Img_Array = []
Img_Label = []

def assign_label(img,flower_type):
    return flower_type

def make_train_data(flower_type,DIR):
    for img in tqdm(os.listdir(DIR)):
        label=assign_label(img,flower_type)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (224,224))
        
        Img_Array.append(np.array(img))
        Img_Label.append(str(label))

In [ ]:
make_train_data('Daisy',daisy_flower_dir)
make_train_data('Sunflower',sunflower_dir)
make_train_data('Tulip',tulip_flower_dir)
make_train_data('Dandelion',dandelion_flower_dir)
make_train_data('Rose',rose_flower_dir)

In [ ]:
# Check visualization random images
fig,ax=plt.subplots(3,4)
fig.set_size_inches(12,12)
for i in range(3):
    for j in range (4):
        l=rn.randint(0,len(Img_Label))
        ax[i,j].axis('off')
        ax[i,j].imshow(Img_Array[l])
        ax[i,j].set_title('Flowers : '+Img_Label[l])

plt.tight_layout()
plt.show()

In [ ]:
# label
le=LabelEncoder()
Y=le.fit_transform(Img_Label)
Y=to_categorical(Y,5)
X=np.array(Img_Array)
X=X/255

# setting random seeds
np.random.seed(42)
rn.seed(42)
tf.random.set_seed(42)

# split train and validation sets
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.20,random_state=42)

In [ ]:
batch_size=128
epochs=100
lr = 0.001

datagen = ImageDataGenerator(
          featurewise_center=False,
          samplewise_center=False,
          featurewise_std_normalization=False,
          samplewise_std_normalization=False,
          zca_whitening=False,
          rotation_range=10,
          zoom_range = 0.1,
          width_shift_range=0.2,
          height_shift_range=0.2,
          horizontal_flip=True,
          vertical_flip=False)

datagen.fit(x_train)

In [ ]:
# modelling CNN.
model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = (6,6),padding = 'Same',activation ='relu', input_shape = (224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(filters =64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(5, activation = "softmax"))

In [ ]:
model.compile(optimizer=Adam(lr),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
History = model.fit(datagen.flow (x_train,y_train, batch_size=batch_size),
          epochs = epochs, validation_data = (x_test,y_test),
          verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)

In [ ]:
import joblib
joblib.dump(model, '/content/gdrive/MyDrive/flowers/model_flowers_v1.pkl')

In [ ]:
plt.plot(History.history['accuracy'])
plt.plot(History.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
pred=model.predict(x_test)
pred_digits=np.argmax(pred,axis=1)

In [ ]:
#from keras.preprocessing import image
from tensorflow.keras.preprocessing import image

In [ ]:
from PIL import Image

In [ ]:
LOAD_MODEL = joblib.load('/content/gdrive/MyDrive/flowers/model_flowers_v1.pkl')

In [ ]:
img = tf.keras.utils.load_img("/content/gdrive/MyDrive/flowers/tulip/112428919_f0c5ad7d9d_n.jpg", target_size = (224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)
img /= 255.
plt.imshow(img[0])                           
plt.axis('off')
plt.show()
preds = LOAD_MODEL.predict(img)
pred_digits=np.argmax(preds,axis=1)
labels = str(le.inverse_transform(pred_digits))
print("PREDICTION       :",labels.replace('[','').replace(']','').replace(']',''))
predictions = preds
score = preds[0]
print("CONFIDENCE SCORE : {0:.5f} %".format(100 * np.max(score)))


In [ ]:
img = tf.keras.utils.load_img("/content/gdrive/MyDrive/flowers/rose/14408977935_a397e796b8_m.jpg", target_size = (224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)
img /= 255.
plt.imshow(img[0])                           
plt.axis('off')
plt.show()
preds = LOAD_MODEL.predict(img)
pred_digits=np.argmax(preds,axis=1)
labels = str(le.inverse_transform(pred_digits))
print("PREDICTION       :",labels.replace('[','').replace(']','').replace(']',''))
predictions = preds
score = preds[0]
print("CONFIDENCE SCORE : {0:.5f} %".format(100 * np.max(score)))


In [ ]:
img = tf.keras.utils.load_img("/content/gdrive/MyDrive/flowers/daisy/25360380_1a881a5648.jpg", target_size = (224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)
img /= 255.
plt.imshow(img[0])                           
plt.axis('off')
plt.show()
preds = LOAD_MODEL.predict(img)
pred_digits=np.argmax(preds,axis=1)
labels = str(le.inverse_transform(pred_digits))
print("PREDICTION       :",labels.replace('[','').replace(']','').replace(']',''))
predictions = preds
score = preds[0]
print("CONFIDENCE SCORE : {0:.5f} %".format(100 * np.max(score)))


In [ ]:
img = tf.keras.utils.load_img("/content/gdrive/MyDrive/flowers/135994133_4f306fe4bf_n.jpg", target_size = (224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)
img /= 255.
plt.imshow(img[0])                           
plt.axis('off')
plt.show()
preds = LOAD_MODEL.predict(img)
pred_digits=np.argmax(preds,axis=1)
labels = str(le.inverse_transform(pred_digits))
print("PREDICTION       :",labels.replace('[','').replace(']','').replace(']',''))
predictions = preds
score = preds[0]
print("CONFIDENCE SCORE : {0:.5f} %".format(100 * np.max(score)))


In [ ]:
img = tf.keras.utils.load_img("/content/gdrive/MyDrive/flowers/141340262_ca2e576490.jpg", target_size = (224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)
img /= 255.
plt.imshow(img[0])                           
plt.axis('off')
plt.show()
preds = LOAD_MODEL.predict(img)
pred_digits=np.argmax(preds,axis=1)
labels = str(le.inverse_transform(pred_digits))
print("PREDICTION       :",labels.replace('[','').replace(']','').replace(']',''))
predictions = preds
score = preds[0]
print("CONFIDENCE SCORE : {0:.5f} %".format(100 * np.max(score)))


In [ ]:
pred = LOAD_MODEL.predict(x_test)
pred_digits=np.argmax(pred,axis=1)

In [ ]:
pred_digits

In [ ]:
print(model.evaluate(x_train, y_train))